In [3]:
!pip3 install mlxtend

In [30]:
from mlxtend.classifier import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


import pandas as pd
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split

In [48]:
data_frame = pd.read_csv("water_potability.csv")
data_labels = data_frame["Potability"]

features = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
       'Organic_carbon', 'Trihalomethanes', 'Turbidity']
data = data_frame[features]

x_train , x_test , y_train , y_test = train_test_split( data , data_labels , test_size = 0.3 )


x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

In [49]:
x_train.ph.fillna(value = x_train.ph.mean() , inplace = True)
x_train.Sulfate.fillna(value = x_train.Sulfate.mean() , inplace = True)
x_train.Trihalomethanes.fillna(value = x_train.Trihalomethanes.mean() , inplace = True)


x_test.ph.fillna(value = x_test.ph.mean() , inplace = True)
x_test.Sulfate.fillna(value = x_test.Sulfate.mean() , inplace = True)
x_test.Trihalomethanes.fillna(value = x_test.Trihalomethanes.mean() , inplace = True)

In [50]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2293, 9)
(983, 9)
(2293,)
(983,)


In [51]:
x_train.isnull().sum()

ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
dtype: int64

In [52]:
x_test.isnull().sum()

ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
dtype: int64

In [16]:
#### now we are taking "svc" , "random_forest" and "GRADIENT_Desent_classifier as base models" and Logistic regression model as
#### meta model

In [21]:
clf1 = SVC( C = 1 , gamma = 'scale' , kernel = "rbf")   ## base model
clf2 = RandomForestClassifier( n_estimators = 150 , criterion = "entropy" , max_depth = 9 )     ## base model
clf3 = GradientBoostingClassifier(max_depth = 4, n_estimators = 150)    ## base model
clf4 = LogisticRegression()     ## meta classifier

In [22]:
sclf = StackingClassifier( classifiers = [ clf1 , clf2 , clf3] , meta_classifier = clf4)

for clf,label in zip([ clf1 , clf2 , clf3 , clf4 ] , [ "SVC" , "RF" , "GBDT" , "LR"]):
    scores = model_selection.cross_val_score(clf , x_train , y_train , cv = 3 , scoring = "accuracy" )
    print("accuracy: %0.2f ( +- %0.2f) [%s] "% ( scores.mean() , scores.std() , label))

accuracy: 0.60 ( +- 0.00) [SVC] 
accuracy: 0.65 ( +- 0.01) [RF] 
accuracy: 0.64 ( +- 0.01) [GBDT] 
accuracy: 0.61 ( +- 0.01) [LR] 


C:\Users\shri\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
sclf.fit( x_train , y_train )
predicted_y = sclf.predict(x_test)

accuracy = accuracy_score( y_test , predicted_y )

print("accuracy is {}".format(accuracy))
##plot_confusion_matrix(sclf , x_test , y_test)

accuracy is 0.6510681586978637


In [33]:
### lets load featurised data and then perform classification on that

In [34]:
data2 = pd.read_csv("featurised_data.csv")

In [35]:
data2.head(5)

,ph,Hardness,Solids,Chloramines,Conductivity,Organic_carbon,Turbidity,Sulfate,Trihalomethanes,ph2,Sulphate2,Trihalomethane
0,4.048281e-14,0.259155,-0.139450,0.112398,1.708693,-1.180470,-1.286101,9.612107e-01,1.305950,2.381253,-0.559402,-0.228051
1,-2.288990e+00,-2.036103,-0.385928,-0.307647,2.062260,0.270556,0.684113,-3.460058e-14,-0.638383,-0.419819,1.787078,-0.228051
2,6.927620e-01,0.847535,-0.240011,1.360386,-0.094018,0.780998,-1.167187,-3.460058e-14,0.001509,-0.419819,1.787078,-0.228051
3,8.408220e-01,0.547568,0.000493,0.591917,-0.778711,1.254943,0.848282,6.394214e-01,2.152545,-0.419819,-0.559402,-0.228051
4,1.368360e+00,-0.464358,-0.460178,-0.363642,-0.343886,-0.824231,0.138764,-6.540767e-01,-2.181266,-0.419819,-0.559402,-0.228051


In [42]:
x_train2 , x_test2 , y_train2 , y_test2 = train_test_split( data2 , data_labels , test_size = 0.3 )


x_train2 = pd.get_dummies(x_train2)
x_test2 = pd.get_dummies(x_test2)



In [43]:
clf1 = SVC( C = 1 , gamma = 'scale' , kernel = "rbf")   ## base model
clf2 = RandomForestClassifier( n_estimators = 150 , criterion = "entropy" , max_depth = 9 )     ## base model
clf3 = GradientBoostingClassifier(max_depth = 4, n_estimators = 150)    ## base model
clf4 = LogisticRegression()     ## meta classifier


sclf = StackingClassifier( classifiers = [ clf1 , clf2 , clf3] , meta_classifier = clf4)

for clf,label in zip([ clf1 , clf2 , clf3 , clf4 ] , [ "SVC" , "RF" , "GBDT" , "LR"]):
    scores = model_selection.cross_val_score(clf , x_train2 , y_train2 , cv = 3 , scoring = "accuracy" )
    print("accuracy: %0.2f ( +- %0.2f) [%s] "% ( scores.mean() , scores.std() , label))

accuracy: 0.67 ( +- 0.00) [SVC] 
accuracy: 0.66 ( +- 0.01) [RF] 
accuracy: 0.64 ( +- 0.01) [GBDT] 
accuracy: 0.61 ( +- 0.00) [LR] 


In [45]:
sclf.fit( x_train2 , y_train2 )
predicted_y = sclf.predict(x_test2)

accuracy = accuracy_score( y_test2 , predicted_y )

print("accuracy is {}".format(accuracy))
##plot_confusion_matrix(sclf , x_test , y_test)

accuracy is 0.62970498474059


In [46]:
### using probability values as meta features

In [47]:
clf1 = SVC( C = 1 , gamma = 'scale' , kernel = "rbf")   ## base model
clf2 = RandomForestClassifier( n_estimators = 150 , criterion = "entropy" , max_depth = 9 )     ## base model
clf3 = GradientBoostingClassifier(max_depth = 4, n_estimators = 150)    ## base model
clf4 = LogisticRegression()     ## meta classifier


sclf = StackingClassifier( classifiers = [ clf1 , clf2 , clf3] , meta_classifier = clf4 , use_probas = True)

for clf,label in zip([ clf1 , clf2 , clf3 , clf4 ] , [ "SVC" , "RF" , "GBDT" , "LR"]):
    scores = model_selection.cross_val_score(clf , x_train2 , y_train2 , cv = 3 , scoring = "accuracy" )
    print("accuracy: %0.2f ( +- %0.2f) [%s] "% ( scores.mean() , scores.std() , label))

accuracy: 0.67 ( +- 0.00) [SVC] 
accuracy: 0.66 ( +- 0.01) [RF] 
accuracy: 0.64 ( +- 0.01) [GBDT] 
accuracy: 0.61 ( +- 0.00) [LR] 


In [54]:
clf1 = SVC( C = 1 , gamma = 'scale' , kernel = "rbf")   ## base model
clf2 = RandomForestClassifier( n_estimators = 150 , criterion = "entropy" , max_depth = 9 )     ## base model
clf3 = GradientBoostingClassifier(max_depth = 4, n_estimators = 150)    ## base model
clf4 = LogisticRegression()     ## meta classifier


sclf = StackingClassifier( classifiers = [ clf1 , clf2 , clf3] , meta_classifier = clf4 ,use_probas = True)

for clf,label in zip([ clf1 , clf2 , clf3 , clf4 ] , [ "SVC" , "RF" , "GBDT" , "LR"]):
    scores = model_selection.cross_val_score(clf , x_train , y_train , cv = 3 , scoring = "accuracy" )
    print("accuracy: %0.2f ( +- %0.2f) [%s] "% ( scores.mean() , scores.std() , label))

accuracy: 0.61 ( +- 0.00) [SVC] 
accuracy: 0.65 ( +- 0.01) [RF] 
accuracy: 0.64 ( +- 0.00) [GBDT] 
accuracy: 0.61 ( +- 0.00) [LR] 
